<a href="https://colab.research.google.com/github/vbanai/Reinforcement-Learning/blob/main/Finding_the_sortest_path_QLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q LEARNING PROJECT</br>
In this section I apply Q learning technique to find the optimal way for the warehouse robot. I downloaded a warehouse picture from bird eye view and edited for this experiment. I split all potential path to sections with equal distances, and placed two products P1 and P2 on the site which the robot should collect. The robot enter the site at landmark A and leave the site through landmark X. The task is to collect the products through the shortest path between A and X. Product P1 can be collected in point N, and P2 product can be collected in point P and point L.


In [132]:
%%html
<iframe src="https://drive.google.com/file/d/1hF99vex70k4XtvbEcOpZzJFt7YRlMLNj/preview" width="640" height="380" allow="autoplay"></iframe>


In [1]:
import numpy as np
gamma = 0.75
alpha = 0.9

# DEFINING THE ENVIRONMENT ACCORDING TO THE WAREHOUSE ABOVE

# Defining the states
location_to_state = {'A': 0,
                     'B': 1,
                     'C': 2,
                     'D': 3,
                     'E': 4,
                     'F': 5,
                     'G': 6,
                     'H': 7,
                     'I': 8,
                     'J': 9,
                     'K': 10,
                     'L': 11,
                     'M': 12,
                     'N': 13,
                     'O': 14,
                     'P': 15,
                     'Q': 16,
                     'R': 17,
                     'X': 18}

# Defining the actions
actions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

# Defining the rewards   row = state,  column = action lead to the next state , 
#  for example "1" in the first row means that only one action is possible from state A and
# this action leads to state "B", so I put reward 1 here. 

            #  A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,X
R = np.array([[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #A
              [1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #B
              [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #C
              [0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0], #D
              [0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], #E
              [0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0], #F
              [0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0], #G
              [0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0], #H
              [0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0], #I
              [0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0], #J
              [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0], #K
              [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0], #L
              [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0], #M
              [0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0], #N
              [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0], #O
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0], #P
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1], #Q
              [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0], #R
              [0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1]])#X
              
              
              
            

#BUILDING THE AI SOLUTION WITH Q-LEARNING

# Making a mapping from the states to the locations
state_to_location = {state: location for location, state in location_to_state.items()}

# Making a function that returns the shortest route from a starting to ending location
def route(starting_location, ending_location):
    R_new = np.copy(R)
    ending_state = location_to_state[ending_location] # convert the letter to number using the inverse dict
    R_new[ending_state, ending_state] = 1000  # the ending location is the top priority location and setting up the reward 1000
    Q = np.array(np.zeros([19,19]))
    for i in range(1000):
        current_state = np.random.randint(0,19)
        playable_actions = []
        for j in range(19):
            if R_new[current_state, j] > 0:
                playable_actions.append(j)
        next_state = np.random.choice(playable_actions)
        TD = R_new[current_state, next_state] + gamma * Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state]
        Q[current_state, next_state] = Q[current_state, next_state] + alpha * TD
    route = [starting_location]
    next_location = starting_location
    while (next_location != ending_location):
        starting_state = location_to_state[starting_location]
        next_state = np.argmax(Q[starting_state,])
        next_location = state_to_location[next_state]
        route.append(next_location)
        starting_location = next_location
    return route


Finding the shortest way while collecting P1 and P2 products

In [3]:
def best_route(starting_location, intermediary_location, ending_location, intermediary_location2=None):
  if intermediary_location2==None:
    return route(starting_location, intermediary_location) + route(intermediary_location, ending_location)[1:]
  else:
    return route(starting_location, intermediary_location) + route(intermediary_location, intermediary_location2)[1:]+route(intermediary_location2, ending_location)[1:]

best_route('A','H','X','P')

['A', 'B', 'C', 'E', 'F', 'G', 'H', 'N', 'O', 'P', 'O', 'Q', 'X']

Finding the shortest way while collecting P1 product

In [4]:
best_route('A','L','X')

['A', 'B', 'D', 'J', 'K', 'L', 'M', 'R', 'Q', 'X']